### Solutions

In [1]:
from datetime import datetime

import pytz
from dateutil import parser
import requests
from pprint import pprint


#### Question 1

Use the `pytz` and `dateutil` libraries to convert this string into a UTC naive `datetime` object.

In [2]:
t = "Feb 8, 2021 5:30pm (Denver Time)"

def to_utc_naive_denver(timestamp: str) -> datetime:
    """Convert a Denver local time string to a UTC naive datetime.

    The input string is expected to look like:
    'Feb 8, 2021 5:30pm (Denver Time)'.
    """
    # Strip the parenthetical portion, which is not useful for parsing
    cleaned = timestamp.split("(")[0].strip()

    # Parse into a timezone-naive datetime using dateutil.parser
    local_dt = parser.parse(cleaned)

    # Attach the America/Denver timezone using pytz
    denver_tz = pytz.timezone("America/Denver")
    if local_dt.tzinfo is None:
        local_dt = denver_tz.localize(local_dt)
    else:
        # In case the parser picked up a timezone, normalize it to Denver first
        local_dt = local_dt.astimezone(denver_tz)

    # Convert to UTC
    utc_dt = local_dt.astimezone(pytz.UTC)

    # Return a UTC *naive* datetime (no tzinfo attached)
    return utc_dt.replace(tzinfo=None)

utc_naive = to_utc_naive_denver(t)
print(utc_naive, type(utc_naive))


2021-02-09 00:30:00 <class 'datetime.datetime'>


#### Question 2

Use the `requests` library to load the following html page:

In [3]:
url = 'https://en.wikipedia.org/wiki/John_von_Neumann'

def fetch_html(url: str) -> str:
    """Return the HTML content for a given URL using a GET request.

    Uses a User-Agent header and *does not* raise an exception on HTTP errors,
    so the rest of the notebook can still run.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; exam-solution/1.0)"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
    except requests.RequestException as exc:
        print(f"Network error while fetching {url}: {exc}")
        return ""

    # If status is not OK, print a warning but don't crash
    if not (200 <= response.status_code < 300):
        print(f"Warning: got HTTP status {response.status_code} for {url}")
        # You may still get some useful HTML in the body:
        return response.text or ""

    return response.text


def extract_title(html: str) -> str:
    """Extract the contents of the <title> tag from an HTML string.

    Uses str.find as required.
    """
    if not html:
        raise ValueError("Empty HTML passed to extract_title().")

    html_lower = html.lower()
    start_tag = "<title>"
    end_tag = "</title>"

    start_idx = html_lower.find(start_tag)
    if start_idx == -1:
        raise ValueError("<title> tag not found in HTML")

    end_idx = html_lower.find(end_tag, start_idx)
    if end_idx == -1:
        raise ValueError("</title> tag not found in HTML")

    content_start = start_idx + len(start_tag)
    title = html[content_start:end_idx].strip()
    return title


html = fetch_html(url)
try:
    page_title = extract_title(html)
    print(page_title)
except Exception as e:
    print(f"Failed to extract title: {e}")


John von Neumann - Wikipedia


Once you have loaded that page, extract the title of that page, which is the text located between the `<title>` and `</title>` tags.

Hint: You'll want to read the Python docs for the `find` method available for strings:

https://docs.python.org/3/library/stdtypes.html#str.find

#### Question 3

Use a `GET` request to this URL:

In [4]:
url = "https://httpbin.org/json"

# Make the GET request
response = requests.get(url, timeout=10)
response.raise_for_status()

content_type = response.headers.get('Content-Type', '')
print(f"Content-Type: {content_type}")

if 'json' in content_type:
    payload = response.json()
else:
    payload = response.json()

print("\nParsed JSON payload:")
pprint(payload)


Content-Type: application/json

Parsed JSON payload:
{'slideshow': {'author': 'Yours Truly',
               'date': 'date of publication',
               'slides': [{'title': 'Wake up to WonderWidgets!', 'type': 'all'},
                          {'items': ['Why <em>WonderWidgets</em> are great',
                                     'Who <em>buys</em> WonderWidgets'],
                           'title': 'Overview',
                           'type': 'all'}],
               'title': 'Sample Slide Show'}}


Use the response from that request to:
- determine the response format
- extract the response into a Python object

#### Question 4

Use a `POST` request to call this url:

In [5]:
url = "https://httpbin.org/anything"

Make this call passing the following query parameters: `a=1` and `b=2`

Also, pass this dictionary as the body of the post request:

In [6]:
data = {
    'x': 100,
    'y': 200,
    'z': ['a', 'b', 'c']
}


Load the returned JSON into a Python object and print it out.

In [14]:
# params = {'a': 1, 'b': 2}
params = {}

try:
    # POST request with query params and JSON body
    response = requests.post(url, params=params, json=data, timeout=10)
    response.raise_for_status()  # raises HTTPError on 4xx/5xx

    # Load the returned JSON into a Python object
    result = response.json()

    # Pretty-print the resulting Python object
    pprint(result)

except requests.Timeout:
    print(f"Request to {url} timed out.")
except requests.HTTPError as exc:
    print(f"HTTP error: {exc}")
except requests.RequestException as exc:
    print(f"Request failed: {exc}")
except ValueError as exc:
    print(f"Could not decode JSON response: {exc}")


{'args': {},
 'data': '{"x": 100, "y": 200, "z": ["a", "b", "c"]}',
 'files': {},
 'form': {},
 'headers': {'Accept': '*/*',
             'Accept-Encoding': 'gzip, deflate',
             'Content-Length': '42',
             'Content-Type': 'application/json',
             'Host': 'httpbin.org',
             'User-Agent': 'python-requests/2.32.5',
             'X-Amzn-Trace-Id': 'Root=1-693b2693-140d5f763928bb364bfb7d15'},
 'json': {'x': 100, 'y': 200, 'z': ['a', 'b', 'c']},
 'method': 'POST',
 'origin': '2.56.12.240',
 'url': 'https://httpbin.org/anything'}
